In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,jalu,LY,2021-08-02 00:54:14,29.0331,21.5482,83.44,36,0,9.06,clear sky
1,1,hualmay,PE,2021-08-02 00:54:15,-11.0964,-77.6139,63.77,83,17,10.78,few clouds
2,2,hithadhoo,MV,2021-08-02 00:49:34,-0.6000,73.0833,81.32,72,85,6.42,overcast clouds
3,3,mount isa,AU,2021-08-02 00:54:02,-20.7333,139.5000,85.77,32,40,8.05,scattered clouds
4,4,mataura,NZ,2021-08-02 00:54:16,-46.1927,168.8643,45.48,83,100,2.10,overcast clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_prompt_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_prompt_temp = float(input('What is the maximum temperature you would like for your trip? '))  

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
min_max_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_prompt_temp) &(city_data_df["Max Temp"] >= min_prompt_temp)]
min_max_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,jalu,LY,2021-08-02 00:54:14,29.0331,21.5482,83.44,36,0,9.06,clear sky
2,2,hithadhoo,MV,2021-08-02 00:49:34,-0.6000,73.0833,81.32,72,85,6.42,overcast clouds
6,6,hilo,US,2021-08-02 00:54:16,19.7297,-155.0900,79.79,78,95,4.00,overcast clouds
8,8,hobyo,SO,2021-08-02 00:54:17,5.3505,48.5268,76.89,78,10,23.02,clear sky
21,21,krumovgrad,BG,2021-08-02 00:54:03,41.4667,25.6500,75.04,52,0,4.94,clear sky


In [17]:
# 4a. Determine if there are any empty rows.
min_max_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
dropped_rows_df = min_max_cities_df.dropna()
dropped_rows_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,jalu,LY,2021-08-02 00:54:14,29.0331,21.5482,83.44,36,0,9.06,clear sky
2,2,hithadhoo,MV,2021-08-02 00:49:34,-0.6000,73.0833,81.32,72,85,6.42,overcast clouds
6,6,hilo,US,2021-08-02 00:54:16,19.7297,-155.0900,79.79,78,95,4.00,overcast clouds
8,8,hobyo,SO,2021-08-02 00:54:17,5.3505,48.5268,76.89,78,10,23.02,clear sky
21,21,krumovgrad,BG,2021-08-02 00:54:03,41.4667,25.6500,75.04,52,0,4.94,clear sky
...,...,...,...,...,...,...,...,...,...,...,...
681,681,vostok,RU,2021-08-02 00:58:11,46.4856,135.8833,78.82,56,94,2.04,overcast clouds
685,685,saint anthony,US,2021-08-02 00:58:13,45.0205,-93.2180,76.82,46,40,3.00,scattered clouds
686,686,bathsheba,BB,2021-08-02 00:58:13,13.2167,-59.5167,81.21,83,40,11.50,scattered clouds
688,688,porlamar,VE,2021-08-02 00:58:14,10.9580,-63.8491,84.29,84,7,9.22,clear sky


In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = dropped_rows_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,jalu,LY,83.44,clear sky,29.0331,21.5482,
2,hithadhoo,MV,81.32,overcast clouds,-0.6000,73.0833,
6,hilo,US,79.79,overcast clouds,19.7297,-155.0900,
8,hobyo,SO,76.89,clear sky,5.3505,48.5268,
21,krumovgrad,BG,75.04,clear sky,41.4667,25.6500,
25,tiarei,PF,82.42,clear sky,-17.5333,-149.3333,
30,karacakoy,TR,77.04,clear sky,41.4029,28.3834,
32,balikpapan,ID,79.79,scattered clouds,-1.2675,116.8289,
35,atuona,PF,77.04,scattered clouds,-9.8000,-139.0333,
38,haines junction,CA,82.35,few clouds,60.7522,-137.5108,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [26]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,jalu,LY,83.44,clear sky,29.0331,21.5482,صاحبة الفخامة
2,hithadhoo,MV,81.32,overcast clouds,-0.6000,73.0833,Scoop Guest House
6,hilo,US,79.79,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
8,hobyo,SO,76.89,clear sky,5.3505,48.5268,Halane Hotel and Restaurent
21,krumovgrad,BG,75.04,clear sky,41.4667,25.6500,Hotel Divna


In [28]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""



In [30]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))